# Assignment 7
* Leon Sixt
* Benjamin Wild

Note: We implemented the whole SIFT paper and also PCA-SIFT [1] to improve the results after the perspective transformation. 

[1] Ke, Yan, and Rahul Sukthankar. "PCA-SIFT: A more distinctive representation for local image descriptors." Computer Vision and Pattern Recognition, 2004. CVPR 2004. Proceedings of the 2004 IEEE Computer Society Conference on. Vol. 2. IEEE, 2004.

# Utility functions
[Jump to the interesting parts](#scale_space)

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import solve
import base64
import math

from joblib import Parallel, delayed

from skimage.filters import gaussian_filter, scharr_h, scharr_v
from skimage.feature import peak_local_max
from skimage.transform import resize, rotate
import skimage.color
import skimage.io

import glob
from sklearn.decomposition import PCA

import scipy.misc
from scipy.signal import convolve
from scipy.spatial.distance import euclidean

import itertools
from itertools import tee, islice
import imageio

from dotmap import DotMap

from IPython.display import HTML

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import seaborn as sns
from more_itertools import pairwise
from collections import OrderedDict
import pickle
sns.set(color_codes=True)

In [ ]:
def plot_sift_circles(keypoints_pyr, orientation_pyr=None, figure=None, ):
    if figure is None:
        figure=plt
        
    for s, keypoints in enumerate(scale_points_pyramid(keypoints_pyr)):
        x, y = keypoints[:, 2], keypoints[:, 1]
        radius = 5*(s+1) + 2*s**2
        for i, p in enumerate(keypoints):
            circle = plt.Circle((p[2], p[1]), radius, color='r', fill=False, lw=1.5)
            figure.add_artist(circle)
            
            if orientation_pyr is not None:
                theta = orientation_pyr[s][i]
                x = np.sin(theta) * radius + p[2]
                y = np.cos(theta) * radius + p[1]
                line = plt.Line2D((p[2], x), (p[1], y), color='r')
                figure.add_artist(line)
            
def scale_points_pyramid(points_pyr):
    return [scale_points(points, s) for s, points in enumerate(points_pyr)]
        
def scale_points(points, s):
    return np.concatenate([points[:, :1], points[:, 1:3] * (2 ** s)], axis=1)

In [ ]:
def plot_initial_keypoints(sift, results):
    fig_before, axes_before = plt.subplots(1, sift.num_octaves, figsize=(16, 4))
    fig_after, axes_after = plt.subplots(1, sift.num_octaves, figsize=(16, 4))
    
    acc_keypoints_pyr = results.acc_keypoint_pyr
    keypoints_pyr = results.initial_keypoint_pyr
    for i, (keypoints, acc_keypoints) in enumerate(zip(keypoints_pyr, acc_keypoints_pyr)):
        axes_before[i].imshow(sift.scaled_images[i], cmap='gray')
        axes_after[i].imshow(sift.scaled_images[i], cmap='gray')
        axes_before[i].scatter(keypoints[:, 2], keypoints[:, 1])
        axes_after[i].scatter(acc_keypoints[:, 2], acc_keypoints[:, 1])
        axes_before[i].axis('off')
        axes_after[i].axis('off')
        
    fig_before.suptitle("Initial keypoints", fontsize=22)
    fig_after.suptitle("Keypoints after taylor approximation and contrast filter", fontsize=22)
    
def plot_orientation(sift, results):
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    fig.suptitle('Keypoints before and after orientation calculation', fontsize=22)

    axes[0].imshow(sift.scaled_images[0], cmap='gray')
    axes[1].imshow(sift.scaled_images[0], cmap='gray')

    plot_sift_circles(results.filtered_keypoint_pyr, figure=axes[0])
    plot_sift_circles(results.oriented_keypoint_pyr, results.orientation_pyr,  figure=axes[1])

    axes[0].axis('off')
    axes[1].axis('off')
    
def plot_octave(index, figsize=(16, 8)):
    lena = load_lena()
    dog, gs = dog_pyramid_octave(lena, index)
    
    images = len(gs)
    fig, axes = plt.subplots(2, images, figsize=figsize)
    for i, g in enumerate(gs):
        axes[0, i].imshow(g, cmap='gray')
        axes[0, i].axis('off')
        if i >= 1:
            axes[1, i].imshow(dog[i-1], cmap='gray')
        axes[1, i].axis('off')
        
    fig.suptitle('Octave {}'.format(index), fontsize=22)

In [ ]:
def get_sigma(scale):
    if scale == 0:
        return (np.sqrt(2)) # *(octave+1)
    else:
        return get_sigma(scale-1)*np.sqrt(2)

def print_sigmas(nb_scales=5):
    for s in range(nb_scales):
        sigma = get_sigma(s)
        print("{:.4f}   ".format(sigma), end='')
    print()
    
def dog_pyramid_octave(im, octave, nb_scales=5):
    gaussians = []
    for s in range(nb_scales):
        sigma = get_sigma(s)
        g = gaussian_filter(im, sigma, mode='mirror')
        gaussians.append(g)
    return np.stack([g1 - g0 for g0, g1 in pairwise(gaussians)]), np.stack(gaussians)

def dog_pyramid(im, nb_octaves=3, nb_layers=5):
    prev = im
    dogs = []
    for i in range(nb_octaves):
        dog, gaussians = dog_pyramid_octave(prev, i, nb_layers)
        prev = gaussians[-1, ::2, ::2]
        dogs.append(dog)
    return dogs

In [ ]:
def load_lena(path=None):
    if path is None:
        lena = scipy.misc.lena() / 255
    else:
        lena = scipy.misc.imread(path) / 255
    return resize(lena, np.array(lena.shape)*2)

In [ ]:
def print_num_keypoints(keypoints_pyr):
    num_keypoints_pyr = list(map(len, keypoints_pyr))
    print(num_keypoints_pyr)
    print(sum(num_keypoints_pyr))
def print_result_keys(results):
    print(list(results.keys()))

<a id='scale_space'></a>
# Scale Space

In [ ]:
plot_octave(0)
plot_octave(5)

# SIFT Implementation
[Jump to the results](#results)

In [ ]:
def get_partial_derivative(tensor, i):
    derivf = np.array([-1, 0, 1])
    derivf = derivf / np.sum(np.abs(derivf))
    return convolve(tensor, derivf.reshape(np.roll([1, 1, 3], i)), mode='same')

def get_derivatives(dogs):
    # partial derivatives of sigma, y and x
    derivs = np.array([get_partial_derivative(dogs, i) for i in range(3)])
    
    H = np.zeros((3, 3, *dogs.shape))
    for indices in itertools.product(range(3), repeat=2):
        d0 = derivs[indices[0]]
        H[indices[0], indices[1]] = get_partial_derivative(d0, indices[1])
        
    return H, derivs

def get_neighbors(file):
    im = skimage.color.rgb2grey(scipy.misc.imread(file))
    im = im / np.max(im)
    sift = SIFT(im)
    try:
        results = sift.get_features(descriptor_type='none')
        padded_size = 39 // 2 
        results.neigbhorhood_keypoint_pyr, results.neighborhood_pyr = \
            sift.get_neighborhoods(results.oriented_keypoint_pyr, results.orientation_pyr, padded_size)  
        neighbours = list(filter(lambda l: len(l) > 0, results.neighborhood_pyr))
    except ValueError as err:
        import traceback
        traceback.print_exc()
        pass
    if len(neighbours) == 0:
        return []
    else:
        return np.concatenate(neighbours)

def fit_sift_pca(directory, num_pca_components=20):
    """Fits *.jpg files in `directory`."""
    all_neighbours = Parallel(n_jobs=-1)(delayed(get_neighbors)(file) for file in \
                                        glob.glob(directory + "/*.jpg"))
    all_neighbours = list(filter(lambda l: len(l) > 0, all_neighbours))
    all_neighbours =  np.concatenate(all_neighbours)
    pca = PCA(num_pca_components)
    pca.fit(all_neighbours.reshape((len(all_neighbours), -1)))
    return pca

class SIFT:
    def __init__(self, image, num_octaves=4, pca=None):
        self.image = image
        self.num_octaves = num_octaves
        self.pca = pca
        
        self.scaled_images = [self.image]
        self.dog_pyramid = []
        self.gaussian_pyramid = []
        self.hessians = []
        self.partial_derivatives = []
        self.image_derivatives = []
        self._build_dogs()
        self._build_dog_derivatives()
        self._build_image_derivatives()
        
    def _build_dogs(self):
        assert len(self.dog_pyramid) == 0
        for i in range(self.num_octaves):
            dogs, gs = dog_pyramid_octave(self.scaled_images[i], i)
            assert dogs.shape[1:] == self.scaled_images[i].shape
            self.dog_pyramid.append(dogs)
            self.gaussian_pyramid.append(gs)
            if i < self.num_octaves - 1:
                self.scaled_images.append(gs[2, ::2, ::2])
        
    def _build_dog_derivatives(self):
        assert len(self.hessians) == 0
        for dogs in self.dog_pyramid:
            H, d = get_derivatives(dogs)
            self.hessians.append(H)
            self.partial_derivatives.append(d)
            
    def _build_image_derivatives(self):
        assert len(self.image_derivatives) == 0
        for image in self.scaled_images:
            self.image_derivatives.append(np.array(np.gradient(image)))
            assert self.image_derivatives[-1][0].shape == image.shape
            
    def get_inital_keypoints(self):
        def extrema(dogs):
            return np.concatenate([peak_local_max(dogs, min_distance=1, threshold_rel=0.001),
                                   peak_local_max(-dogs, min_distance=1, threshold_rel=0.001)])
        
        return [extrema(dogs) for dogs in self.dog_pyramid]
    
    def get_features(self, descriptor_type='sift'):
        results = DotMap()
        results.initial_keypoint_pyr = self.get_inital_keypoints()
        results.acc_keypoint_pyr = self.get_accurate_keypoints(results.initial_keypoint_pyr)
        results.filtered_keypoint_pyr = self.eliminate_edge_responses(results.acc_keypoint_pyr)
        results.oriented_keypoint_pyr, results.orientation_pyr = \
            self.assign_orientations(results.filtered_keypoint_pyr)
        
        if descriptor_type == 'sift':
            wsize = 16 // 2
            dsize = 4 // 2
            padded_size = wsize + dsize
            results.neigbhorhood_keypoint_pyr, results.neighborhood_pyr = \
                self.get_neighborhoods(results.oriented_keypoint_pyr, results.orientation_pyr, padded_size)    
            results.descriptor_point_pyr, results.descriptor_pyr = \
                self.get_sift_descriptors(results.neigbhorhood_keypoint_pyr, results.neighborhood_pyr)
        elif descriptor_type == 'pca-sift':
            assert(self.pca is not None)
            padded_size = 39 // 2 
            results.neigbhorhood_keypoint_pyr, results.neighborhood_pyr = \
                self.get_neighborhoods(results.oriented_keypoint_pyr, results.orientation_pyr, padded_size)  
            results.descriptor_point_pyr, results.descriptor_pyr = \
                self.get_pca_sift_descriptors(results.neigbhorhood_keypoint_pyr, results.neighborhood_pyr)
        return results
        
    def get_accurate_keypoints(self, keypoint_pyr):
        acc_keypoints_pyr = []
        for i, keypoints in enumerate(keypoint_pyr):
            acc_keypoints = self._get_accurate_keypoints(keypoints, i)
            acc_keypoints_pyr.append(acc_keypoints)
        return acc_keypoints_pyr

    def _get_accurate_keypoints(self, extrema, pyramid_level, finalized_extrema=None, num_iteration=0):
        def shift(e, offset):
            return np.round(e + offset).astype('int')

        def in_bounds(p):
            try:
                H[:, :, p[0], p[1], p[2]]
                return True
            except IndexError:
                return False

        def has_sufficent_constrast(point, offset, threshold=0.03):
            p = point
            D = dogs[tuple(point)]
            d = derivs[:, p[0], p[1], p[2]]
            contrast = D + 0.5 * d.T @ offset
            return np.abs(contrast) > threshold

        def is_extrema(point, offset, max_iterations=20):
            shift_point = shift(point, offset) 
            return (num_iteration >= max_iterations or \
                np.all(shift_point == point) or \
                np.min(shift_point) < 0) and \
                in_bounds(shift_point)               

        def get_taylor_offset(p):
            h = H[:, :, p[0], p[1], p[2]]
            d = derivs[:, p[0], p[1], p[2]]
            return -np.linalg.pinv(h) @ d.T
        
        H = self.hessians[pyramid_level]
        derivs = self.partial_derivatives[pyramid_level]
        dogs = self.dog_pyramid[pyramid_level]

        if finalized_extrema is None:
            finalized_extrema = []
        if len(extrema) == 0:
            return np.array(finalized_extrema)

        modified_extrema = []
        for point in extrema:
            offset = get_taylor_offset(point)
            if is_extrema(point, offset):
                if has_sufficent_constrast(point, offset):
                    finalized_extrema.append(point)
            else:
                shift_point = shift(point, offset) 
                if in_bounds(shift_point):
                    modified_extrema.append(shift_point)       
        if not modified_extrema:
            return np.array(finalized_extrema)
        return self._get_accurate_keypoints(modified_extrema, pyramid_level, finalized_extrema, num_iteration + 1)  
    
    def eliminate_edge_responses(self, keypoint_pyr, r=10.):
        remaining_point_pyr = []
        for pyramid_level, keypoints in enumerate(keypoint_pyr):
            H = self.hessians[pyramid_level][:2, :2]
            
            remaining_points = []
            for point in keypoints:
                h = H[:, :, point[0], point[1], point[2]]
                c = (np.trace(h)**2) / np.linalg.det(h)
                
                if c < (r + 1)**2 / r:
                    remaining_points.append(point)        
            remaining_point_pyr.append(np.array(remaining_points))
        return remaining_point_pyr
    
    def assign_orientations(self, keypoint_pyr, num_bins=36):
        def get_window_center(slicex, slicey, point):
            def get_center(sl):
                return np.argmin(np.abs(np.array(range(sl.start, sl.stop))))
            return get_center(slicex) + point[1], get_center(slicey) + point[2]

        oriented_keypoint_pyr = []
        orientation_pyr = []
        for pyramid_level, (keypoints, image) in enumerate(zip(keypoint_pyr, self.image_derivatives)):
            oriented_keypoints = []
            orientations = []
            for point in keypoints:
                sigma = 1.5 * get_sigma(point[0])
                wsize = int(sigma * 3 / 2)

                sx = slice(max(0, point[1] - wsize), min(image.shape[1], point[1] + wsize + 1))
                sy = slice(max(0, point[2] - wsize), min(image.shape[2], point[2] + wsize + 1))

                gradx = image[0, sx, sy]
                grady = image[1, sx, sy]

                centerx, centery = get_window_center(sx, sy, point)

                gradient = np.sqrt(gradx**2 + grady**2)
                theta = np.arctan2(grady, gradx)

                gaussianx = scipy.stats.norm(centerx, sigma).pdf(range(sx.start, sx.stop))
                gaussiany = scipy.stats.norm(centery, sigma).pdf(range(sy.start, sy.stop))
                gaussian = gaussianx[:, np.newaxis] @ gaussiany[:, np.newaxis].T            

                bins, _ = np.histogram(theta, weights=gaussian * gradient, bins=num_bins, range=(-np.pi, np.pi))

                highest_peak = max(bins)
                indices = np.nonzero(bins >= 0.8 * highest_peak)[0]

                for idx in indices:
                    x = np.array(range(idx-1, idx+2))
                    y = bins[x % num_bins] 

                    coeffs = np.polyfit(x, y, deg=2)
                    parabola_fit = -coeffs[1] / (2 * coeffs[0])
                    assert(not(np.any(np.isnan(parabola_fit))))

                    oriented_keypoints.append(point)
                    orientations.append((parabola_fit - 0.5 * num_bins) / (0.5 * num_bins) * np.pi)

            oriented_keypoint_pyr.append(np.array(oriented_keypoints))
            orientation_pyr.append(np.array(orientations))
        return oriented_keypoint_pyr, orientation_pyr
    
    def get_neighborhoods(self, keypoint_pyr, orientation_pyr, padded_size):
        def in_bounds(p, image):
            try:
                image[p[0], p[1]]
                if np.min(p) >= 0:
                    return True
                return False
            except IndexError:
                return False

        def get_rotated_subwindows(image, orientation, sx, sy, ds):
            rotated = rotate(image[sx, sy], orientation/np.pi*180)
            assert(rotated.shape == image[sx, sy].shape)
            gradx, grady = np.gradient(rotated[ds, ds])
            return gradx, grady

        rotbox_size = int(np.ceil(padded_size * np.sqrt(2)))
                
        descriptor_pyr = []
        point_pyr = []
        for pyramid_level, (keypoints, orients, octave) in \
                enumerate(zip(keypoint_pyr, orientation_pyr, self.gaussian_pyramid)):
            descriptors = []
            points = []
            for point, point_orientation in zip(keypoints, orients):
                image = octave[point[0]]
                assert(not(np.any(np.isnan(image))))                
                sx = slice(point[1] - rotbox_size, point[1] + rotbox_size + 1)
                sy = slice(point[2] - rotbox_size, point[2] + rotbox_size + 1)
                
                if np.any([not(in_bounds(p, image)) for p in ((sx.start, sy.start), (sx.stop - 1, sy.stop - 1))]):
                    continue

                ds = slice(rotbox_size - padded_size, rotbox_size + padded_size)
                
                gradx, grady = get_rotated_subwindows(image, -point_orientation, sx, sy, ds)
                descriptors.append(np.stack((gradx, grady)))
                points.append(point)
            if descriptors:
                descriptor_pyr.append(np.stack(descriptors))
            else:
                descriptor_pyr.append([])
            
            point_pyr.append(np.array(points))
        return point_pyr, descriptor_pyr
    
    def get_sift_descriptors(self, keypoint_pyr, gradient_pyr, num_bins=8, wsize=16//2, dsize=4):       
        def get_odd_linear_kernel(size):
            return np.array([max(0, (1 - abs(d) / size)) for d in np.arange(-size, size+1, 1)])[:, np.newaxis]

        def get_even_linear_kernel(size):
            return np.array([max(0, (1 - abs(d) / size)) for d in np.arange(-size+0.5, size, 1)])[:, np.newaxis]
        
        bin_kernel = get_even_linear_kernel(dsize) @ get_even_linear_kernel(dsize).T
        
        def normalize(vec):
            vec = vec / np.linalg.norm(vec)
            vec = np.minimum(0.2, vec)
            return vec / np.linalg.norm(vec)

        hist_kernel = get_odd_linear_kernel(num_bins//2)
        
        padded_size = wsize + dsize // 2
        gaussianx = scipy.stats.norm(0, wsize).pdf(range(-padded_size, padded_size))
        gaussiany = scipy.stats.norm(0, wsize).pdf(range(-padded_size, padded_size))
        gaussian = gaussianx[:, np.newaxis] @ gaussiany[:, np.newaxis].T
        
        descriptor_pyr = []
        point_pyr = []
        for pyramid_level, (keypoints, gradients, octave) in \
                enumerate(zip(keypoint_pyr, gradient_pyr, self.gaussian_pyramid)):
            descriptors = []
            points = []         
            for point, gradient in zip(keypoints, gradients):
                gradx = gradient[0]
                grady = gradient[1]

                magnitude = np.sqrt(gradx**2 + grady**2) * gaussian
                theta = np.arctan2(grady, gradx)

                point_descriptors = []
                for fromx in np.arange(0, wsize*2, dsize):
                    for fromy in np.arange(0, wsize*2, dsize):

                        sx = slice(fromx, fromx+2*dsize)
                        sy = slice(fromy, fromy+2*dsize)

                        window_magnitude = magnitude[sx, sy] * bin_kernel
                        window_theta = theta[sx, sy]

                        bins, _ = np.histogram(window_theta, weights=window_magnitude, 
                                               bins=num_bins, range=(-np.pi, np.pi))
                        bins = scipy.ndimage.convolve(bins, hist_kernel.flatten(), mode='wrap')
                        
                        point_descriptors.append(bins)
                point_descriptors = np.concatenate(point_descriptors)
                descriptors.append(normalize(point_descriptors))
                points.append(point)
            descriptor_pyr.append(np.array(descriptors))
            point_pyr.append(np.array(points))
        return point_pyr, descriptor_pyr
    
    def get_pca_sift_descriptors(self, keypoint_pyr, gradient_pyr):
        def process_pyramid_level(keypoints, gradients):
            gradients = np.stack(gradients).reshape(len(gradients), -1)
            descriptors = self.pca.transform(gradients)
            return keypoints, descriptors
        
        point_pyr, descriptor_pyr = zip(*list(map(lambda x: process_pyramid_level(*x), 
                                             zip(keypoint_pyr, gradient_pyr))))
            
        return point_pyr, descriptor_pyr

<a id='results'></a>
# Keypoint detection, taylor approximation and contrast filter

In [ ]:
lena = load_lena()
sift = SIFT(lena)
lena_results = sift.get_features(descriptor_type='sift')

In [ ]:
plot_initial_keypoints(sift, lena_results)

In [ ]:
print_num_keypoints(lena_results.initial_keypoint_pyr)
print_num_keypoints(lena_results.filtered_keypoint_pyr)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

fig.suptitle('Keypoints before and after taylor approximation and contrast filter', fontsize=22)

axes[0].imshow(lena, cmap='gray')
axes[1].imshow(lena, cmap='gray')

plot_sift_circles(lena_results.initial_keypoint_pyr, figure=axes[0])
plot_sift_circles(lena_results.acc_keypoint_pyr, figure=axes[1])

axes[0].axis('off')
axes[1].axis('off')

# Edge response elimination

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Keypoints before and after edge response elimination', fontsize=22)

axes[0].imshow(lena, cmap='gray')
axes[1].imshow(lena, cmap='gray')

plot_sift_circles(lena_results.acc_keypoint_pyr, figure=axes[0])
plot_sift_circles(lena_results.filtered_keypoint_pyr, figure=axes[1])

axes[0].axis('off')
axes[1].axis('off')

# Orientation calculation and parabola fit

In [ ]:
plot_orientation(sift, lena_results)

# Keypoint matching

In [ ]:
lena_transformed = load_lena('Lenna_transformed.png')

In [ ]:
plt.imshow(lena_transformed, cmap=plt.cm.gray)
_ = plt.axis('off')

In [ ]:
transformed_sift = SIFT(lena_transformed)

In [ ]:
transformed_results = transformed_sift.get_features()

In [ ]:
def plot_orientation(sift, results, figsize=(16, 8)):
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('Keypoints before and after orientation calculation', fontsize=22)

    axes[0].imshow(sift.scaled_images[0], cmap='gray')
    axes[1].imshow(sift.scaled_images[0], cmap='gray')

    plot_sift_circles(results.filtered_keypoint_pyr, figure=axes[0])
    plot_sift_circles(results.oriented_keypoint_pyr, results.orientation_pyr,  figure=axes[1])

    axes[0].axis('off')
    axes[1].axis('off')
    
plot_orientation(sift, lena_results)
plt.show()
plot_orientation(transformed_sift, transformed_results)
plt.show()

In [ ]:
def sift_matching(first, second, threshold=0.7):
    first_descriptors = np.concatenate(first.descriptor_pyr)
    second_descriptors = np.concatenate(second.descriptor_pyr)
    
    matches = OrderedDict()
    distances = np.zeros((first_descriptors.shape[0], second_descriptors.shape[0]))
    for i, first_descriptor in enumerate(first_descriptors):
        distances[i] = np.linalg.norm(second_descriptors - first_descriptor, axis=1)
    
    while True:
        def unravel(idx):
            return np.unravel_index(idx, distances.shape)

        def ratio(distances):
            return distances[0] / distances[1]
        
        rows = np.argpartition(distances, 2, axis=1)[:, :2]
        dists = np.array([(distances[r, rows[r, 0]], distances[r, rows[r, 1]]) for r in range(len(rows))])
        ratios = dists[:, 0] / dists[:, 1]
        ratios[np.isnan(ratios)] = np.inf
        bestidx = np.argmin(ratios)
        
        if ratios[bestidx] < threshold:
            i = bestidx
            j = rows[bestidx][0]
            distances[i, :] = np.inf
            distances[:, j] = np.inf
            matches[i] = j
        else:
            break
            
    return matches

In [ ]:
def plot_matching_dict(first_sift, second_sift, first_results, second_results, matching_dict, nbest=25):
    def matching_points(matching_dict):
        first_descriptors = np.concatenate(scale_points_pyramid(first_results.descriptor_point_pyr))
        second_descriptors = np.concatenate(scale_points_pyramid(second_results.descriptor_point_pyr))
        first_index = np.array(list(matching_dict.keys()))
        second_index = np.array(list(matching_dict.values()))
        return list(zip(first_descriptors[first_index], second_descriptors[second_index]))
    
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    ax1.axis('off')
    ax2.axis('off')
    ax1.imshow(first_sift.scaled_images[0], cmap=plt.cm.gray)
    ax2.imshow(second_sift.scaled_images[0], cmap=plt.cm.gray)
    plot_sift_circles(first_results.oriented_keypoint_pyr, first_results.orientation_pyr, figure=ax1)
    plot_sift_circles(second_results.oriented_keypoint_pyr, second_results.orientation_pyr, figure=ax2)
    
    assert np.min(np.concatenate(second_results.descriptor_point_pyr)) >= 0
    def matching_line(p1, p2):
        con = lambda: ConnectionPatch([p1[2], p1[1]], [p2[2], p2[1]],  coordsA="data", coordsB="data", 
                                       arrowstyle="<-", alpha=0.75, ec=sns.color_palette()[4], lw=3,
                                       axesA=ax2, axesB=ax1)
        ax2.add_artist(con())
    
    for f, s in islice(matching_points(matching_dict), nbest):
        matching_line(s, f)

In [ ]:
matching_dict = sift_matching(lena_results, transformed_results)
plot_matching_dict(sift, transformed_sift, lena_results, transformed_results, matching_dict, nbest=10)
plt.show()

# Keypoint matching using PCA-SIFT

In [ ]:
pca = fit_sift_pca('pca_train_images')

In [ ]:
with open("pca.pickle", "wb") as f:
    pickle.dump(pca, f)

In [ ]:
trans_pca_sift = SIFT(lena_transformed, pca=pca)
trans_pca_res = trans_pca_sift.get_features('pca-sift')
pca_lena = SIFT(lena, pca=pca)
pca_lena_res = pca_lena.get_features('pca-sift')

In [ ]:
matching_dict = sift_matching(pca_lena_res, trans_pca_res)
plot_matching_dict(pca_lena, trans_pca_sift, pca_lena_res, trans_pca_res, matching_dict)
plt.show()